In [1]:
import nltk
import re
import codecs
import csv
import os
from sklearn import metrics

path = 'D:\Dropbox\ongoing\chrisIntervention'

In [2]:
# read cohort info from the CSV and save them with the sbj ID

intvIdx = 'Ex1'

category = {}
with open(os.path.join(path, intvIdx + '_key.csv')) as f:
    sbjData = csv.reader(f)
    next(sbjData) # skip the header
    for row in sbjData:
        category[row[0]] = int(row[1])

subjectId = category.keys()        
        
print(subjectId)
print()
print('# of cohort 1: ', sum(1 for sbj in category if category[sbj] == 1))
print('# of cohort 2: ', sum(1 for sbj in category if category[sbj] == 2))


dict_keys(['210100_1', '620048_1', '820126_1', '920049_1', '410213_1', '1020134_1', '211904_1', '420055_1', '210306_1', '212100_1', '520085_1', '420037_1', '211002_1', '510621_1', '120002_1', '120017_1', '120064_1', '510109_1', '1120055_1', '320089_1', '212202_1', '920115_1', '211101_1', '320003_1', '110064_1', '910116_1', '1120074_1', '220053_1', '1020069_1', '320038_1', '820094_1', '1110206_1', '510713_1', '710203_1', '520086_1', '920003_1', '420066_1', '520032_1', '420022_1', '710207_1', '1120009_1', '410221_1', '212104_1', '420052_1', '120031_1', '310047_1', '310155_1', '610202_1', '220023_1', '120001_1', '520138_1', '910506_1', '720071_1', '920063_1', '120084_1', '210700_1', '1120126_1', '1020097_1', '520024_1', '1120065_1', '220002_1', '710526_1', '220117_1', '120049_1', '1020101_1', '1120012_1', '810310_1', '710517_1', '110018_1', '510506_1', '520164_1', '920064_1', '1010227_1', '410401_1', '1010225_1', '210401_1', '410122_1', '920085_1', '1110616_1', '520061_1', '610701_1', '11

In [3]:
# build the dictionary based on the training set
# this is just to check if it is working
# we should build the dictionary for each leave-one-out test

def make_dictionary(subjectPool, critNumSbj):
    allWords = []
    for sbj in subjectPool:
        sbjWords = []
        for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
            line = line.strip()
            line = line.lower()
            tokens = nltk.wordpunct_tokenize(line)
            text = nltk.Text(tokens)
            words = [w for w in text if re.search('[a-z]+',w) and len(w)>2]
            sbjWords.extend(words)
        # when making dictionary, make the words from a document unique
        sbjWords = list(set(sbjWords))
        allWords.extend(sbjWords)
    
    # once we make list of all words, we count the words
    cntWords = nltk.FreqDist(allWords)
    #for ii in range(21,0,-2):
    #    print('# words with freq >', ii, ' : ', sum(1 for word in cntWords if cntWords[word] > ii))
    
    # we go with the words that came from at least N different people (arbitrary) 
    # we can change and see what happens ...
    return [word for word in cntWords if cntWords[word] > critNumSbj]

dictionary = make_dictionary(subjectId, 10)

print(dictionary)

['football', 'more', 'own', 'friends', 'was', 'bad', 'well', 'favorite', 'out', 'say', 'fun', 'help', 'hang', 'much', 'really', 'when', 'mad', 'most', 'times', 'why', 'want', 'art', 'down', 'makes', 'after', 'and', 'can', 'had', 'team', 'new', 'just', 'its', 'happy', 'keep', 'stuff', 'day', 'people', 'spending', 'friend', 'not', 'love', 'anything', 'them', 'because', 'your', 'soccer', 'enjoy', 'someone', 'listening', 'have', 'then', 'also', 'sport', 'the', 'without', 'last', 'great', 'helps', 'these', 'thing', 'things', 'living', 'very', 'home', 'for', 'other', 'sad', 'nice', 'way', 'enjoying', 'second', 'think', 'sports', 'what', 'cause', 'need', 'about', 'important', 'see', 'picked', 'funny', 'this', 'going', 'some', 'don', 'values', 'group', 'only', 'were', 'they', 'take', 'feel', 'should', 'music', 'with', 'something', 'around', 'humor', 'are', 'how', 'thats', 'play', 'lot', 'all', 'time', 'first', 'listen', 'always', 'care', 'draw', 'life', 'through', 'tell', 'independent', 'any',

In [4]:
# we define a function that tunrs text into a list of tokens

def extract_feature(text, dictionary):
    text = text.strip()
    text = text.lower()
    tokens = nltk.wordpunct_tokenize(text)
    text = nltk.Text(tokens)
    feature = {}
    for word in dictionary:
        feature[word] = (word in text)
    return feature

print('nltk output: ', nltk.wordpunct_tokenize('You think you can do this one, Kate?'))
print(extract_feature('You think you can do this one, Kate?', dictionary))

nltk output:  ['You', 'think', 'you', 'can', 'do', 'this', 'one', ',', 'Kate', '?']


{'about': False,
 'active': False,
 'after': False,
 'all': False,
 'alot': False,
 'also': False,
 'always': False,
 'and': False,
 'any': False,
 'anything': False,
 'are': False,
 'around': False,
 'art': False,
 'bad': False,
 'basketball': False,
 'because': False,
 'been': False,
 'being': False,
 'best': False,
 'better': False,
 'bored': False,
 'but': False,
 'can': True,
 'care': False,
 'cause': False,
 'could': False,
 'creative': False,
 'day': False,
 'doing': False,
 'don': False,
 'dont': False,
 'down': False,
 'draw': False,
 'enjoy': False,
 'enjoying': False,
 'even': False,
 'every': False,
 'everything': False,
 'family': False,
 'favorite': False,
 'feel': False,
 'first': False,
 'football': False,
 'for': False,
 'friend': False,
 'friends': False,
 'fun': False,
 'funny': False,
 'game': False,
 'get': False,
 'going': False,
 'good': False,
 'great': False,
 'group': False,
 'had': False,
 'hang': False,
 'hanging': False,
 'happy': False,
 'have': False,
 'h

In [5]:
# leave-one-out classification

predLabel = []
trueLabel = []
currSbj = 1;

for sbj in subjectId:
    # sbj is being tested, others are used to train the classifier
    trainSbj = list(subjectId)
    trainSbj.remove(sbj)
    # create the dictionary
    trainDict = make_dictionary(trainSbj, 5)
    trainText = []
    # create the training set
    for tSbj in trainSbj:
        tmpText = ''
        for line in codecs.open(os.path.join(path, intvIdx, tSbj + '.txt'), 'r' , 'utf-8'):
            tmpText = tmpText + ' ' + line
        trainText.append([extract_feature(tmpText, trainDict), category[tSbj]])
    # train classifier
    classifier = nltk.NaiveBayesClassifier.train(trainText)

    # now the test set
    trueLabel.append( category[sbj] )
    testText = ''
    for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
        testText = testText + ' ' + line
    # predict!
    currPred = classifier.classify(extract_feature(testText, trainDict))
    predLabel.append( currPred )
    
    print(str(currSbj) + ': pred - ' + str(currPred) + ', actual - ' + str(category[sbj]) )
    currSbj = currSbj + 1

print('done. ')


1: pred - 1, actual - 1
2: pred - 2, actual - 2
3: pred - 1, actual - 2
4: pred - 1, actual - 2
5: pred - 1, actual - 1
6: pred - 2, actual - 2
7: pred - 2, actual - 1
8: pred - 2, actual - 2
9: pred - 2, actual - 1
10: pred - 2, actual - 1
11: pred - 2, actual - 2
12: pred - 2, actual - 2
13: pred - 1, actual - 1
14: pred - 2, actual - 1
15: pred - 2, actual - 2
16: pred - 2, actual - 2
17: pred - 2, actual - 2
18: pred - 1, actual - 1
19: pred - 2, actual - 2
20: pred - 2, actual - 2
21: pred - 2, actual - 1
22: pred - 2, actual - 2
23: pred - 1, actual - 1
24: pred - 2, actual - 2
25: pred - 1, actual - 1
26: pred - 1, actual - 1
27: pred - 2, actual - 2
28: pred - 2, actual - 2
29: pred - 2, actual - 2
30: pred - 1, actual - 2
31: pred - 2, actual - 2
32: pred - 1, actual - 1
33: pred - 2, actual - 1
34: pred - 1, actual - 1
35: pred - 2, actual - 2
36: pred - 2, actual - 2
37: pred - 1, actual - 2
38: pred - 2, actual - 2
39: pred - 2, actual - 2
40: pred - 2, actual - 1
41: pred 

In [6]:
# ok, it is POSSIBLE TO PREDICT cohort from the writings.

print(metrics.classification_report(trueLabel, predLabel))


             precision    recall  f1-score   support

          1       0.72      0.55      0.62       148
          2       0.71      0.84      0.77       197

avg / total       0.72      0.72      0.71       345



In [9]:
# now, show me the informative features

dictionary = make_dictionary(subjectId, 5)

allText = []

for sbj in subjectId:
    tmpText = ''
    for line in codecs.open(os.path.join(path, intvIdx, sbj + '.txt'), 'r' , 'utf-8'):
        tmpText = tmpText + ' ' + line
        allText.append([extract_feature(tmpText, dictionary), category[sbj]])

# train classifier based on all information
classifier = nltk.NaiveBayesClassifier.train(allText)

# and show me the informative features
classifier.show_most_informative_features(50)




Most Informative Features
                  values = True                1 : 2      =     12.8 : 1.0
              everything = True                1 : 2      =     12.8 : 1.0
                   never = True                1 : 2      =      5.9 : 1.0
                    into = True                1 : 2      =      5.8 : 1.0
                     old = True                1 : 2      =      5.8 : 1.0
                     now = True                1 : 2      =      5.6 : 1.0
                     see = True                1 : 2      =      5.6 : 1.0
                    dont = True                1 : 2      =      5.1 : 1.0
                    such = True                1 : 2      =      4.9 : 1.0
                    same = True                1 : 2      =      4.9 : 1.0
                 writing = True                1 : 2      =      4.9 : 1.0
                 trouble = True                1 : 2      =      4.9 : 1.0
                     rap = True                1 : 2      =      4.9 : 1.0